# TME 6 : Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME5 pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [36]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence


In [37]:
def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    nucList = [nuc[random.randint(0,3)].upper() if upper else nuc[random.randint(0,3)].lower() for i in range(n)]
    sequence = "".join(nucList)
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = motif
    posToModified = random.sample(range(len(motif)), nbpos)
    for pos in posToModified:
        newNuc = random.choice(nuc)
        while newNuc == motif[pos]:
            newNuc = random.choice(nuc)
        motifM = motifM[0:pos] + newNuc + motifM[pos+1:]
    return motifM

def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
    motif = generateRandomSequence(k)
    print("motif: ", motif)
    modifiedMotifs = [modifierMotif(motif, v) for i in range(t)]
    print("modifiedMotifs: ", modifiedMotifs)
    posToInsert = [random.randint(0, n-k-1) for i in range(t)]
    preSequences = [generateRandomSequence(n-k) for i in range(t)]
    sequences = [preSequences[i][:posToInsert[i]+1] + modifiedMotifs[i] + preSequences[i][posToInsert[i]+1:] for i in range(t)]
    return sequences

adn = implantMotifVar(k, v, t, n)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

motif:  ACAGA
modifiedMotifs:  ['ACCGA', 'GCAGA', 'CCAGA']
['TTACCGAACC', 'GGAGCAGAGA', 'GCCCCCAGAG']
['TTACCGAACC', 'GGAGCAGAGA', 'GCCCCCAGAG']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [38]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
	"""
	Génère un vecteur de position aléatoires
	entrée sequences: matrice de dimension txn contenant les sequences
	entrée k: taille du motif
	sortie s: vecteur de position aléatoires, une position par séquence
  REMARQUE les positions doivent être inférieur à n-k
	"""
	return [random.randint(0, len(sequences[i])-k) for i in range(len(sequences))]

s = generateRandomS(adn, k)
print (adn)
print (s)

['TTACCGAACC', 'GGAGCAGAGA', 'GCCCCCAGAG']
[2, 3, 5]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). Attention, utiliser les pseudocount 1 pour éviter les plus tard les probabilités à zéro.


In [39]:
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn avec les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs
    """
    motifs = []
    try:
        motifs = [seqs[i][s[i]:s[i]+k] for i in range(len(seqs))]
    except IndexError:
        pass
    return motifs

motifs = extractSeqs(s, adn, k)
# print(motifs)

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    nDict = {nuc[i]: i for i in range(len(nuc))}
    q = len(nuc)
    mf = np.ones((q, k))
    for col in range(k):
        for line in range(len(motifs)):
            # try:
            nucleotide = motifs[line][col]
            # except IndexError:
                # print(line, col, motifs)
            mf[nDict[nucleotide]][col] += 1
    return mf
    
MF = profile(motifs, k, nuc)
# print(MF)

4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes. Faire aussi une fonction pour calculer la sequence consensus d'une matrice de frequence.

In [40]:
# motifs = [s.upper() for s in motifs]

def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  q = len(nuc)
  colSumList = np.sum(MF, 0)
  colSum = colSumList[0]
  for i in range(1, len(colSumList)):
      assert(colSumList[i] == colSum)
  # MF = np.ones((q, k))
  # print(PWM)
  return np.array([MF[i, j]/colSum for i in range(q) for j in range(k)]).reshape(q, k)

PWM = generatePWM(MF)
print(PWM)

[[0.28571429 0.28571429 0.28571429 0.28571429 0.42857143]
 [0.28571429 0.42857143 0.28571429 0.14285714 0.14285714]
 [0.28571429 0.14285714 0.28571429 0.42857143 0.28571429]
 [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714]]


In [41]:
# si égal ?

def getConsensus(MF, k, nuc):
    """
    Renvoie la sequence consensus d'une matrice de frequence
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie seqCons: sequence consensus
    N.B. en cas d'égalité, on prend la sequence avec les premières nucléotides selon l'ordre de ACGT
    """
    minIndexArray = np.argmax(MF, axis=0)
    return "".join([nuc[index] for index in minIndexArray])

seqCons = getConsensus(MF, k, nuc)
print(seqCons)


ACAGA


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [42]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    pos = 0; prob = 1
    #print (PWM)
    for n in motif:
        i = nuc.index(n)
        #print (i, pos)
        prob *=PWM[i, pos]
        pos +=1
    return prob

prob = probability(PWM, seqCons)
print (prob)

0.006425893972749447


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [43]:
def pMostProbkmer(P, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position du k-mer le plus probable dans la sequence
    """
    probList = [probability(P, sequence[i:i+k]) for i in range(len(sequence)-k+1)]
    maxList = max(probList) # le premier plus grand
    return probList.index(maxList)


s = pMostProbkmer(PWM, k, "GCC" + seqCons+ "GGATACT")
print (s)

3


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [44]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """
    s = [pMostProbkmer(PWM, k, seq) for seq in sequences]
   
    return s

ns = getNewS(PWM, k, adn)
print (ns)

[2, 3, 4]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [45]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    print(P1, P2, np.mat(P1) == np.mat(P2), (np.mat(P1) == np.mat(P2)).all())
    return (np.mat(P1) == np.mat(P2)).all()

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée P: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    mf = np.mat(MF)
    maxArray = np.max(mf, axis=0)
    return np.sum(maxArray)

# print(adn)
# print(seqCons)

# print(MF)
sc = getScore(MF, k)
print (sc)


13.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [46]:
def GreedyProfileMotifSearch(sequences, t, n, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
	  entrée sequences: matrice de dimension txn contenant les séquences 
	  entrée t : nombre de séquences 
	  entrée n : longueur des séquences 
	  entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    s = generateRandomS(sequences, k)
    bestScore = 0
    bestS = []
    while getScore(profile(extractSeqs(s, sequences, k), k, nuc), k) > bestScore:
        bestS = s
        tmpProfile = profile(extractSeqs(s, sequences, k), k, nuc)
        bestScore = getScore(tmpProfile, k)
        s = getNewS(tmpProfile, k, sequences)
    return bestS, int(bestScore)


s, bestScore = GreedyProfileMotifSearch(adn, t, n, k)   
motifs = extractSeqs(s, adn, k)
print (motifs)
        

['GAACC', 'GGAGC', 'GCCCC']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

In [47]:
nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv = implantMotifVar(k, v, t, n)
print("sequences:", mv)

sMv, bestScoreMv = GreedyProfileMotifSearch(mv, t, n, k)
print("s, bestScore:", sMv, bestScoreMv)
seqsMv = extractSeqs(sMv, mv, k)
print(seqsMv)
consensusMv = getConsensus(profile(seqsMv, k, nuc), k, nuc)
print(consensusMv)

motif:  ACGGA
modifiedMotifs:  ['CCGGA', 'AGGGA', 'ACGGC']
sequences: ['GTCCCGGACT', 'TCTGTAGGGA', 'AGCAACGGCT']
s, bestScore: [0, 2, 0] 13
['GTCCC', 'TGTAG', 'AGCAA']
AGCAA


<strong>Reponse:</strong> En exécutant l'algorithme plusieures fois, on constate que parfois on trouve le motif implémenté avec 1 ou 0 variation, et que la plupart du temps on trouve plus de 2 variations, i.e. on ne trouve pas le motif implémenté.

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

<font><strong>Remarque :</strong> Avec cette version d'itération, la plupart du temps, on trouve au moins un motif implémenté avec 1 variation. Mais on peut trouver également le motif original ou celui avec 2 variations. De plus, le bon motif consensu n'a pas forcément la meuilleure Score.</font>

In [48]:
def GreedyProfileMotifSearchIte(sequences, t, n, k, It=50):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
	  entrée séquences: matrice de dimension txn contenant les séquences 
	  entrée t : nombre de séquences 
	  entrée n : longueur des séquences 
	  entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= consensus sequence du motif
    """
    pos = {}
    consensus = {}
    for i in range(It):
        s, Score = GreedyProfileMotifSearch(sequences, t, n, k)
        motifConsensus = getConsensus(profile(extractSeqs(s, sequences, k), k, nuc), k, nuc)
        if Score not in pos.keys():
            pos[Score] = s
            consensus[Score] = motifConsensus
    return pos, consensus

def sortSCK(posDict, consensusDict, rev = True):
    return sorted([(posDict[k], consensusDict[k],k) for k in posDict.keys()], key=lambda item:item[2], reverse = rev)

s, consensus = GreedyProfileMotifSearchIte(adn, t, n, k, 50)
print(s, consensus)


{17: [3, 4, 3], 14: [1, 3, 2], 16: [5, 1, 0], 15: [2, 1, 2], 13: [2, 4, 1], 18: [3, 4, 5]} {17: 'CCGAG', 14: 'CCCCA', 16: 'GAACC', 15: 'ACCCA', 13: 'CCCAA', 18: 'CAGAG'}


In [49]:
nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv2 = implantMotifVar(k, v, t, n)
print("sequences:", mv2)

posDict, consensusDict = GreedyProfileMotifSearchIte(mv2, t, n, k)


print("s, concensus, k: ", sortSCK(posDict, consensusDict))


motif:  GTTTC
modifiedMotifs:  ['GTTTT', 'ATTTC', 'GTTTT']
sequences: ['TTGCGTTTTA', 'GTATATTTCC', 'TGTTTTCATT']
s, concensus, k:  [([4, 0, 1], 'GTTTT', 18), ([4, 5, 1], 'GTTTT', 17), ([5, 2, 3], 'TTTTA', 16), ([1, 2, 0], 'TGATT', 15), ([3, 0, 5], 'CCATT', 14)]


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [50]:
k=7; v=1; t=20; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_2.fasta" #Remplacer par votre fichier

sequencesChip = readFasta(genome, n)
t = len(sequencesChip)
s, consensus = GreedyProfileMotifSearchIte(sequencesChip, t, n, k)
# print(s, concensus)


Afficher les top motifs (leurs sequence consensus) de meilleurs scores.

In [51]:
def printTopMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
      # try:
        print (consensus[motifsSort[i]])
      # except IndexError:
      #     pass

top = 10
printTopMotifsScore(consensus, top)

TTATTTT
TTTTCAT
TTATTTA
TATTTTT
ATAAATT
AGCATTT
TTTCATT
TATATTA
AAGTTTT
AATTTTA


13\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de k à n, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [52]:
def getRandomFixePositions(k, n):
    """
    Genere une projection de k vers n
    entrée k: nombre de positions du motif original
    entrée n: nombre de positions choisi pour la projection 
    sortie projection: liste de positions choisi aléatoirement
    """    
    return random.sample(range(k), n)

lR = getRandomFixePositions(7, 4)
print (lR)

def generateKey(positions, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée positions : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    return "".join(motif[positions[i]] for i in range(len(positions)))

generateKey(lR, "01234567")

[1, 5, 4, 6]


'1546'

14\. Implémenter l'algorithme ``randomProjection``.

In [53]:
def isRepeatN(seq, repeatMax = 5):
    repeatDict = {nu: 0 for nu in nuc}
    for nu in seq:
        repeatDict[nu] += 1
        if repeatDict[nu] >= repeatMax:
            return True
    return False

def checkRepeat2(seq, repeat2Max = 5):
    repeat2s = {(n1+n2)*(repeat2Max//2)+n1*(repeat2Max % 2) for n1 in nuc for n2 in nuc if n1!=n2}
    for repeat2 in repeat2s:
        if seq.find(repeat2) != -1:
            return True
    return False

def randomProjection(k, n, sequences, removeLowComplexe = False):
    """
    Implémente l'algorithme randomProjection
    entrée k : nombre de positions du motif
    entrée n : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    
    motifs  = dict(); motifsSeq  = dict()
    posFixed = getRandomFixePositions(k, n)
    not_valid_motif = set()
    for i in range(len(sequences)):
        seq = sequences[i]
        for j in range(len(seq)-k+1):
            motif =  seq[j:j+k]
            key = generateKey(posFixed, motif)
            if key not in motifs.keys():
                if key not in not_valid_motif:
                    if removeLowComplexe:
                        if not checkRepeat2(motif) and not isRepeatN(motif):
                            motifs[key] = 1
                            motifsSeq[key] = motif
                        else:
                            not_valid_motif.add(key)
                    else:
                        motifs[key] = 1
                        motifsSeq[key] = motif
            else:
                motifs[key] += 1
    return motifs, motifsSeq


adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

def sortPS(motifsProj, motifsSeq, rev = True):
    return sorted([(motifsProj[k], motifsSeq[k], k) for k in motifsProj.keys()], key=lambda item:item[0], reverse = rev)
sortPS(motifsProj, motifsSeq)

{'CATG': 1, 'GAAC': 1, 'GCAA': 1, 'CGCC': 1, 'ATCG': 1, 'CCTC': 1, 'GACC': 1, 'CCAA': 1, 'GCAC': 1, 'GCCC': 1, 'CGCG': 1, 'CGGT': 1}
{'CATG': 'TTAACGG', 'GAAC': 'TAACGGC', 'GCAA': 'AACGGCA', 'CGCC': 'ACGGCAC', 'ATCG': 'GCTCACG', 'CCTC': 'CTCACGC', 'GACC': 'TCACGCC', 'CCAA': 'CACGCCA', 'GCAC': 'TACCGGC', 'GCCC': 'ACCGGCC', 'CGCG': 'CCGGCCG', 'CGGT': 'CGGCCGT'}


[(1, 'TTAACGG', 'CATG'),
 (1, 'TAACGGC', 'GAAC'),
 (1, 'AACGGCA', 'GCAA'),
 (1, 'ACGGCAC', 'CGCC'),
 (1, 'GCTCACG', 'ATCG'),
 (1, 'CTCACGC', 'CCTC'),
 (1, 'TCACGCC', 'GACC'),
 (1, 'CACGCCA', 'CCAA'),
 (1, 'TACCGGC', 'GCAC'),
 (1, 'ACCGGCC', 'GCCC'),
 (1, 'CCGGCCG', 'CGCG'),
 (1, 'CGGCCGT', 'CGGT')]

15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: On peut trouver le motif implémenté, mais il n'a pas forcément la plus grande fréquence, et ce motif est souvent implémenté avec 1 variation. En exécutant plusieurs fois le code, on augmente la chance de le trouvé.

In [54]:
nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv3 = implantMotifVar(k, v, t, n)
print("sequences:", mv3)

projMv3, seqMv3 = randomProjection(k, 3, mv3)
print(sortPS(projMv3, seqMv3))

motif:  CTGCA
modifiedMotifs:  ['CCGCA', 'CTGCT', 'TTGCA']
sequences: ['ACCGCAAACC', 'CCTGCTATTA', 'ATACATTGCA']
[(2, 'CCGCA', 'GAC'), (2, 'CCTGC', 'TCG'), (2, 'CTATT', 'ATT'), (1, 'ACCGC', 'CCG'), (1, 'CGCAA', 'CAA'), (1, 'GCAAA', 'AAA'), (1, 'CAAAC', 'ACA'), (1, 'AAACC', 'ACC'), (1, 'CTGCT', 'GTC'), (1, 'TGCTA', 'CAT'), (1, 'GCTAT', 'TTA'), (1, 'TATTA', 'TAT'), (1, 'ATACA', 'AAC'), (1, 'TACAT', 'CTA'), (1, 'CATTG', 'TGT')]


16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [55]:
def randomProjIt(sequences, k, v, nuc, It=50, removeLowComplexe=False):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    
    motifs = []; scores = []
    for i in range(It):
        motifsProj, motifsSeq = randomProjection(k, v, sequences, removeLowComplexe)
        bestScore, motif, bucket = sortPS(motifsProj, motifsSeq)[0]
        scores.append(bestScore)
        motifs.append(motif)

    return max(scores), motifs[scores.index(max(scores))] 

score, seqsMotif = randomProjIt(adn, 7, 4, nuc, 100)

print (score, seqsMotif)


2 CCCCCAG


17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [56]:
score, seqsMotif = randomProjIt(sequencesChip, 7, 4, nuc, 100)
print(score, seqsMotif)

430 TCTTTCT


18\. Vous avez sans doute remarqué que les algos implémentés trouvent, suivant des motifs peu complexes, réviser les implémentations pour régler/diminuer ce problème.

<strong>réponse : </strong>On ajoute un paramètre optionel `removeLowComplexe`, à défault False, aux fonction `randomProjection` et `randomProjIt`. Si ce paramètre vaut True, alors les motifs simples seront éliminés.<br/>
Donc on obtient 430 TCTTTCT (avant d'éliminer) 378 CGTTTAT (après). (score, motif)


In [57]:
score, seqsMotif = randomProjIt(sequencesChip, 7, 4, nuc, 100, True)
print(score, seqsMotif)

378 CGTTTAT
